# **U-Net Model**

---



In [4]:
from keras.models import *
from keras.layers import *
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy,binary_crossentropy


def weighted_bincrossentropy(true, pred, weight_zero = 0.3, weight_one = 1):


    # calculate the binary cross entropy
    bin_crossentropy = keras.backend.binary_crossentropy(true, pred)

    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weighted_bin_crossentropy = weights * bin_crossentropy
    return keras.backend.mean(weighted_bin_crossentropy)


def unet_model(input_shape=(128, 128, 6)):
    inputs = Input(shape=input_shape)

    # Downsample path
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D((2, 2))(drop4)

    # Bottom
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Upsample path
    up6 = Conv2D(256, (2, 2), activation='relu', padding='same')(UpSampling2D((2, 2))(drop5))
    concat6 = Concatenate()([drop4, up6])
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = Conv2D(128, (2, 2), activation='relu', padding='same')(UpSampling2D((2, 2))(conv6))
    concat7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = Conv2D(64, (2, 2), activation='relu', padding='same')(UpSampling2D((2, 2))(conv7))
    concat8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)


    up9 = Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D((2, 2))(conv8))
    concat9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.00005), loss=weighted_bincrossentropy, metrics=['accuracy'])
    return model

# **Loading Training Data**

---



In [5]:
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
data=np.load('/content/drive/MyDrive/GP/data.npy',allow_pickle=True)
masks=np.load('/content/drive/MyDrive/GP/masks.npy',allow_pickle=True)


# **Training Process**

---



In [ ]:
train_data, val_data, train_masks, val_masks = train_test_split(data, masks, test_size=0.20, random_state=None)

model = unet_model()

history = model.fit(train_data, train_masks, batch_size=8, epochs=20, validation_data=(val_data, val_masks))

# **Patches Testing**

---



In [10]:
total_recall=0
total_Percision=0
total_accuracy=0
total_Miou=0
count=0

In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import sklearn.metrics

for i in test_18_19:
  image1 = cv2.imread('/content/drive/MyDrive/GP/finalData/2018_2019/2018/' +i)
  image2 = cv2.imread('/content/drive/MyDrive/GP/finalData/2018_2019/2019/' +i)


  # Concatenate the two images along the third dimension
  image = np.concatenate((image1, image2), axis=2)

  # Expand dimensions to match the model input shape (1, 128, 128, 6)
  input_image = np.expand_dims(image, axis=0)

  predicted_mask = model.predict(input_image)[0]

  predicted_mask = np.round(predicted_mask)

  mask = np.zeros((128, 128, 3), dtype=np.uint8)

  mask[predicted_mask[:, :, 0] == 1] = (255, 255, 255)

  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)


  #compare part

  act_mask=cv2.imread("/content/drive/MyDrive/GP/finalData/2018_2019/Mask/"+i,0)

  #leave masks with no change
  if np.mean(act_mask)==0:
    continue

  count=count+1
  cv2_imshow(mask)
  cv2_imshow(act_mask)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  acc = sklearn.metrics.accuracy_score(np.array(act_mask), np.array(mask))
  precision = sklearn.metrics.precision_score(np.array(act_mask), np.array(mask),average="weighted")
  recall = sklearn.metrics.recall_score(np.array(act_mask), np.array(mask),average="weighted")
  miou=sklearn.metrics.jaccard_score(np.array(act_mask), np.array(mask),average="micro")
  total_accuracy=total_accuracy+acc*100
  total_recall=total_recall+recall*100
  total_Percision=total_Percision+precision*100
  print("miou",miou*100)
  total_Miou=total_Miou+miou*100
  print("Recall ",recall*100)
  print("Percision ",precision*100)
  print("Accuracy ",acc*100)


NameError: ignored

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

for i in test_19_20:
  image1 = cv2.imread('/content/drive/MyDrive/GP/finalData/2019_2020/2019/' +i)
  image2 = cv2.imread('/content/drive/MyDrive/GP/finalData/2019_2020/2020/' +i)


  image = np.concatenate((image1, image2), axis=2)


  input_image = np.expand_dims(image, axis=0)

  predicted_mask = model.predict(input_image)[0]

  predicted_mask = np.round(predicted_mask)

  mask = np.zeros((128, 128, 3), dtype=np.uint8)

  mask[predicted_mask[:, :, 0] == 1] = (255, 255, 255)
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)


  act_mask=cv2.imread("/content/drive/MyDrive/GP/finalData/2019_2020/Mask/"+i,0)
  if np.mean(act_mask)==0:
    continue

  count=count+1
  cv2_imshow(mask)
  cv2_imshow(act_mask)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  import sklearn.metrics
  acc = sklearn.metrics.accuracy_score(np.array(act_mask), np.array(mask))
  precision = sklearn.metrics.precision_score(np.array(act_mask), np.array(mask),average="weighted")
  recall = sklearn.metrics.recall_score(np.array(act_mask), np.array(mask),average="weighted")
  miou=sklearn.metrics.jaccard_score(np.array(act_mask), np.array(mask),average="micro")
  total_accuracy=total_accuracy+acc*100
  total_recall=total_recall+recall*100
  total_Percision=total_Percision+precision*100
  total_Miou=total_Miou+miou*100
  print("miou",miou*100)
  print("Recall ",recall*100)
  print("Percision ",precision*100)
  print("Accuracy ",acc*100)


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

for i in test_20_21:
  image1 = cv2.imread('/content/drive/MyDrive/GP/finalData/2020_2021/2020/' +i)
  image2 = cv2.imread('/content/drive/MyDrive/GP/finalData/2020_2021/2021/' +i)


  image = np.concatenate((image1, image2), axis=2)

  input_image = np.expand_dims(image, axis=0)

  predicted_mask = model.predict(input_image)[0]

  predicted_mask = np.round(predicted_mask)

  mask = np.zeros((128, 128, 3), dtype=np.uint8)

  mask[predicted_mask[:, :, 0] == 1] = (255, 255, 255)
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)


  act_mask=cv2.imread("/content/drive/MyDrive/GP/finalData/2020_2021/Mask/"+i,0)
  if np.mean(act_mask)==0:
    continue

  count=count+1
  cv2_imshow(mask)
  cv2_imshow(act_mask)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  import sklearn.metrics
  acc = sklearn.metrics.accuracy_score(np.array(act_mask), np.array(mask))
  precision = sklearn.metrics.precision_score(np.array(act_mask), np.array(mask),average="weighted")
  recall = sklearn.metrics.recall_score(np.array(act_mask), np.array(mask),average="weighted")
  miou=sklearn.metrics.jaccard_score(np.array(act_mask), np.array(mask),average="micro")
  total_accuracy=total_accuracy+acc*100
  total_recall=total_recall+recall*100
  total_Percision=total_Percision+precision*100
  total_Miou=total_Miou+miou*100
  print("miou",miou*100)
  print("Recall ",recall*100)
  print("Percision ",precision*100)
  print("Accuracy ",acc*100)



**Metrics Results**

---



In [ ]:
total_accuracy=total_accuracy/count
total_Percision=total_Percision/count
total_recall=total_recall/count
total_Miou=total_Miou/count
print("acuuracy ",total_accuracy)
print("Percision ",total_Percision)
print("Recall ",total_recall)
F1 = 2 * (total_Percision * total_recall) / (total_Percision + total_recall)
print("FSCORE ",F1)
print("Miou",total_Miou)

# **Full Image Testing**

In [ ]:

import cv2
import numpy as np
from google.colab.patches import cv2_imshow


image1 = cv2.imread('/content/drive/MyDrive/GP/Input/image_3.tif')
image2 = cv2.imread('/content/drive/MyDrive/GP/Input/image_4.tif')

height, width, _ = image1.shape


patch_size = 128

stride = patch_size

# Create an empty mask image of the same size as the input images
mask = np.zeros((height, width), dtype=np.uint8)

for y in range(0, height - patch_size + 1, stride):
    for x in range(0, width - patch_size + 1, stride):
        # Extract the patch from the first image
        patch1 = image1[y:y+patch_size, x:x+patch_size, :]

        # Extract the patch from the second image
        patch2 = image2[y:y+patch_size, x:x+patch_size, :]

        concatenated_patch = np.concatenate((patch1, patch2), axis=2)

        normalized_patch = concatenated_patch / 255.0

        input_patch = np.expand_dims(normalized_patch, axis=0)

        predicted_mask = model.predict(input_patch)[0]

        predicted_mask = np.round(predicted_mask)

        mask[y:y+patch_size, x:x+patch_size] = predicted_mask[:, :, 0]


In [ ]:
predicted_mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
predicted_mask[mask != 0] = (255, 255, 255)
predicted_mask = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
print(predicted_mask.shape)

cv2_imshow(predicted_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

**Metrics Results**

---



In [ ]:
act_mask=cv2.imread("/content/drive/MyDrive/GP/CD/dif_3_4.tif",0)

import sklearn.metrics
acc = sklearn.metrics.accuracy_score(np.array(act_mask), np.array(predicted_mask),normalize=True)
precision = sklearn.metrics.precision_score(np.array(act_mask), np.array(predicted_mask),average="weighted")
recall = sklearn.metrics.recall_score(np.array(act_mask), np.array(predicted_mask),average="weighted")
F1=sklearn.metrics.f1_score(np.array(act_mask), np.array(predicted_mask),average="weighted")
Miou=sklearn.metrics.jaccard_score(np.array(act_mask), np.array(predicted_mask),average="micro")
print("Recall ",recall*100)
print("Percision ",precision*100,"")
F1 = 2 * (precision * recall) / (precision + recall)
print("FSCORE ",F1*100)
print("Accuracy ",acc*100)
print("Miou",Miou*100)